In [2]:
import torch
import torch.nn as nn
import sys
import random
from sklearn.model_selection import KFold
import math
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# 设置GPU使用方式
# 获取GPU列表
gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]

if gpus:
    try:
        # 设置每个GPU为增长式占用
        for gpu in gpus:
            torch.cuda.set_per_process_memory_fraction(1.0, gpu)  # 可根据需要调整占用比例
    except RuntimeError as e:
        # 打印异常
        print(e)

In [5]:
class FBCNet(nn.Module):
    def __init__(self, m=32,C=30,Nc=2,T=200,Nb=9,w=50):
        super(FBCNet, self).__init__()
        self.dropout = 0.5
        self.conv1 = nn.Conv2d(Nb, m*Nb, (C, 1), padding=0)
        self.batchnorm1 = nn.BatchNorm2d(m*Nb, False)
        self.fc1 = nn.Linear(12607488,Nc)
        

    def forward(self, x,w=50):
        #Spatial convolution
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.elu(x)
        #Temporal Feature Extration
        x1 = torch.var(x[:,:,:,0:w], dim=3, keepdim=True)
        x2 = torch.var(x[:,:,:,w:2*w], dim=3, keepdim=True)
        x3 = torch.var(x[:,:,:,2*w:3*w], dim=3, keepdim=True)
        x4 = torch.var(x[:,:,:,3*w:4*w], dim=3, keepdim=True)
        x=torch.cat((x1,x2,x3,x4),dim=3)
        x = F.logsigmoid(x)
        print('log',x.shape)
        #   Classifier
        x = torch.flatten(x)
        print('faltten',x.shape)
        x = self.fc1(x)  # 使用线性层
        x = F.softmax(x)
        print(x.shape)
        return x

    def apply_max_norm(self, max_norm=2.0):
        torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm)

In [6]:
#导入数据
#rest1
datapath1=r'D:\JQ_YJS\过山车实验数据epochs转一个class（4）\rest1_class.npy' 
data1=np.load(datapath1)
print(data1.shape)
#rest2
datapath2=r'D:\JQ_YJS\过山车实验数据epochs转一个class（4）\rest1_class.npy'
data2=np.load(datapath2)
print(data2.shape)
data_all=np.concatenate((data1,data2),axis=0)

(1650, 30, 200)
(1650, 30, 200)


In [7]:
#标签制作
label_all = torch.cat([torch.zeros(10505), torch.ones(10505)]).long()  # 标签：前10505个为0，后10505个为1

In [9]:
# 创建十折交叉验证
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# 保存结果的列表
historys = []
test_pred = []
test_real = []
accuracy, precision, recall, f1score = [], [], [], []
batchsz = 16

# 进行十折交叉验证
for fold, (train_ind, test_ind) in enumerate(kfold.split(data_all, label_all)):
    print('fold号:', fold + 1)

    # 每一折验证前都要打乱训练集样本顺序
    n = len(train_ind)
    A = np.linspace(0, n - 1, n, dtype=int)
    random.shuffle(A)

    # 构建训练集、验证集、测试集
    epoch_train = data_all[train_ind[A[:int(0.8 * n)]]]
    epoch_val = data_all[train_ind[A[int(0.8 * n):]]]
    epoch_test = data_all[test_ind]
    label_train = label_all[train_ind[A[:int(0.8 * n)]]]
    label_val = label_all[train_ind[A[int(0.8 * n):]]]
    label_test = label_all[test_ind]

    # 转换为Tensor并创建DataLoader
    train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))
    
    train_loader = DataLoader(train_dataset, batch_size=batchsz, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batchsz, shuffle=False)

    # 选择、创建模型
    model = FBCNet()
    print(model)

    # 配置模型训练
    criterion = nn.CrossEntropyLoss()  # 使用交叉熵损失
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # 开始训练模型
    print('开始训练!!')
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(100):  # 训练100个epoch
        model.train()
        running_loss = 0.0

        # 训练阶段
        for inputs, targets in train_loader:
            
            print(f"训练批次输入数据形状: {inputs.shape}")
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        # 验证阶段
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                print('啦啦啦',input.shape)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        history['val_loss'].append(avg_val_loss)

    # 保存训练记录
    historys.append(history)

    # 计算、保存测试结果
    model.eval()
    with torch.no_grad():
        pred_test = model(torch.tensor(epoch_test, dtype=torch.float32))
        pred_test = torch.argmax(pred_test, dim=1).numpy()

    # 保存预测结果和真实结果
    test_pred.append(pred_test)
    test_real.append(label_test)

    # 计算准确率，精确率，召回率，F1评分
    acc = accuracy_score(label_test, pred_test)
    pre = precision_score(label_test, pred_test, average='macro')
    rec = recall_score(label_test, pred_test, average='macro')
    f1 = f1_score(label_test, pred_test, average='macro')
    
    accuracy.append(acc)
    precision.append(pre)
    recall.append(rec)
    f1score.append(f1)

    print(f"$$ 测试集准确率为 accuracy: {acc}")
    print(f"$$ 测试集精确率为 precision: {pre}")
    print(f"$$ 测试集召回率为 recall: {rec}")
    print(f"$$ 测试集 F1 评分为 f1_score: {f1}")

# 将每一折 history 中误差结果保存（训练集和测试集，用于反映训练过程）    
loss_train = []
loss_val = []
for history_s in historys:
    loss_val.append(history_s['val_loss'])
    loss_train.append(history_s['train_loss'])

fold号: 1
FBCNet(
  (conv1): Conv2d(9, 288, kernel_size=(30, 1), stride=(1, 1))
  (batchnorm1): BatchNorm2d(288, eps=False, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=12607488, out_features=2, bias=True)
)
开始训练!!
训练批次输入数据形状: torch.Size([16, 30, 200])


C:\Users\FJQ\AppData\Local\Temp\ipykernel_24440\2139316532.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_24440\2139316532.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


RuntimeError: Given groups=1, weight of size [288, 9, 30, 1], expected input[1, 16, 30, 200] to have 9 channels, but got 16 channels instead